In [38]:
import pandas as pd
import json

In [3]:
emosim_data = json.loads(open('./emoji_datasets/EmoSim508.json').read())
'''
json format
{
    'emojiPair': 
        {
        'emojiOne': 
            {'title': 'musical note',
                'unicodelong': '\\U0001F3B5',
                'unicodeshort': 'U+1F3B5'},
       'emojiTwo': 
           {'title': 'musical notes',
            'unicodelong': '\\U0001F3B6',
            'unicodeshort': 'U+1F3B6'}},
    'emojiPairId': '199',
    'emojiPairSimilarity': 
      {'Google_Sense_All': 1.0,
       'Google_Sense_Def': 1.0,
       'Google_Sense_Desc': 0.84,
       'Google_Sense_Label': 0.94,
       'Human_Annotator_Agreement': 4.0,
       'Twitter_Sense_All': 1.0,
       'Twitter_Sense_Def': 1.0,
       'Twitter_Sense_Desc': 0.89,
       'Twitter_Sense_Label': 0.92}
}
Pandas format:
Emoji,1,2,...10
\\U0001F3B5',\\U0001F3B6',...,null
'''

In [33]:
#example of information we want: 
emoji_unicode = emosim_data[0]['emojiPair']['emojiOne']['unicodelong']
human_rating = emosim_data[0]['emojiPairSimilarity']['Human_Annotator_Agreement']

In [34]:
print(emoji_unicode,human_rating)

\U0001F3B5 4.0


In [80]:
def extract_pairs(emosim_json):
    # converts emosim json to list of tuples: (emoji1, emoji2, similarity)
    # returns dictionary 
    # dictionary entry looks like this: 
    # emoji1 : [(emoji2, similarity),(other pairs),...]
    pairs = {}
    for pair in emosim_json:
        emoji1 = pair['emojiPair']['emojiOne']['unicodelong']
        emoji2 = pair['emojiPair']['emojiTwo']['unicodelong']
        similarity = pair['emojiPairSimilarity']['Human_Annotator_Agreement']
        if emoji1 not in pairs: 
            pairs[emoji1] = [(emoji2,similarity)]
        else: 
            pairs[emoji1].append((emoji2,similarity))
        if emoji2 not in pairs: 
            pairs[emoji2] = [(emoji1,similarity)]
        else: 
            pairs[emoji2].append((emoji1,similarity))
    #Sort the rankings
    for emoji in pairs.keys():
        similar_emojis = pairs[emoji]
        sorted_emojis = sorted(similar_emojis, key=lambda tup: tup[1])
        pairs[emoji] =  sorted_emojis    
    return pairs

def emoji_to_ASCII(string):
    # example input: '😀hi'
    # example output: '\\U0001f600hi'
    return string.encode('unicode-escape').decode('ASCII')

def ASCII_to_emoji(string):
    # example input: '\\U0001f600hi'
    # example output: '😀hi'
    return string.encode('ASCII').decode('unicode-escape')

In [74]:
emoji_to_ASCII('🇬🇧')

'\\U0001f1ec\\U0001f1e7'

In [73]:
ASCII_to_emoji('\\U0001F1EC_\\U0001F1E7')
# \U0001F3B5

'🇬_🇧'

In [82]:
emoji_rankings = extract_pairs(emosim_data)
emoji_rankings.keys()

dict_keys(['\\U0001F3B5', '\\U0001F3B6', '\\U0001F38A', '\\U0001F389', '\\U0000263A', '\\U0001F60A', '\\U00002764', '\\U0001F49E', '\\U0001F495', '\\U0001F498', '\\U0001F60D', '\\U0001F49C', '\\U0001F499', '\\U0001F497', '\\U0001F493', '\\U0001F3A4', '\\U0001F496', '\\U0001F49A', '\\U0001F49B', '\\U0001F3A7', '\\U0001F3CC', '\\U000026F3', '\\U0001F3BC', '\\U0001F388', '\\U0001F605', '\\U0001F602', '\\U0001F618', '\\U0001F601', '\\U0001F62D', '\\U0001F629', '\\U0001F44D', '\\U0001F44C', '\\U0001F62B', '\\U0001F622', '\\U0001F48B', '\\U0001F381', '\\U0001F382', '\\U0001F606', '\\U0001F644', '\\U0001F914', '\\U0001F4AB', '\\U00002728', '\\U0001F494', '\\U0001F604', '\\U00002705', '\\U0001F614', '\\U0001F449', '\\U0001F448', '\\U0001F44F', '\\U0001F648', '\\U0001F64A', '\\U0001F61D', '\\U0001F609', '\\U00002714', '\\U0001F4FB', '\\U000027A1', '\\U00002B05', '\\U0001F4AF', '\\U0001F30A', '\\U0001F3C4', '\\U0001F63B', '\\U0001F64C', '\\U0001F64F', '\\U0001F60C', '\\U0001F440', '\\U0001F649',

In [68]:
# emoji_rankings[emoji_to_ASCII('😀')]
emoji_rankings['\\U0001f600']

TypeError: 'dict_values' object is not subscriptable